# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science 

##  Project F: Measuring the Shape and Brightness of Galaxies with Neural Networks
### Ideation 5
### Group 75: Dmitry Vukolov, Ning Xu, Rohit Beri, Sunil Chomal


**Harvard University**<br/>
**Spring 2020**<br/>
**Instructors**: Pavlos Protopapas, Mark Glickman and Chris Tanner<br/>


<hr style="height:2pt">

---



In [1]:
# RUN THIS CELL
import requests
from IPython.core.display import HTML, display

styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css"
).text
HTML(styles)

## <div class="theme"> Outline </div> 

1. Overview
2. Loading the Data
3. Structure and Size of the Data
4. Basic Statistics of the Data 
5. Visualization of the Image Data 
6. Distributions of the Labels and Numerical Attributes 
7. Parametric Image Generation
8. Summary

## <div class="theme"> Overview </div> 

<div style="border: 3px solid #800080; padding: 10px"> 

## Creating Galaxy images using GalSim

## <div class="theme"> Part 1 </div> 

<div style="border: 3px solid #800080; padding: 10px"> 

## GalSim Images with fixed Sigma and PSF

### <div class='exercise'> <b> Step 0: Load the essential libraries </b> </div>

In [2]:
# Load useful libraries
import os
import sys
import copy
import math
import time
import random
import numpy as np
import pandas as pd
import ipywidgets as widgets

# Import h5py for reading h5 files
import h5py

# Load galsim for data generation
import galsim

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Tweak plot resolution and styling
%config InlineBackend.figure_format = "retina"
sns.set(style="white", palette=None, rc={"axes.linewidth": 1})
plt.rc("image", cmap="viridis")

In [3]:
# Load useful libraries
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import EarlyStopping

### <div class='exercise'> <b> Step 1: Load the dataset with fixed noise and fixed PSF </b> </div>

In [4]:
# Read the data set
dataset = "./cs109b-project/data/data_v1.npz"

with np.load(dataset) as data:
    print("Available variables:", data.files)

    image = data["img"]
    image_nonoise = data["img_nonoise"]
    label = data["label"]
    snr = data["snr"]
    sigma = data["sigma"]
    psf = data["psf_r"]

Available variables: ['img', 'img_nonoise', 'label', 'psf_r', 'snr', 'sigma', 'train_test']


<div class='gc'> <b> Join Sigma and PSF with the Label data </b> </div>

In [5]:
# Combine Singma and PSF with the label
label = np.hstack([label, sigma.reshape(-1,1), psf.reshape(-1,1)])

### <div class='exercise'> <b> Step 2: Normalize the data and split the data into Traning and Validation sets </b> </div>

In [6]:
# Reshape images to 3D array
image = image.reshape(image.shape[0], image.shape[1], image.shape[2], 1)
image_nonoise = image_nonoise.reshape(image_nonoise.shape[0], image_nonoise.shape[1], image_nonoise.shape[2], 1)


# Split into 90% train and 10% test (creates a view of the array)
n_train = int(label.shape[0] * 0.9)
image_train, image_val = image[:n_train], image[n_train:]
image_nonoise_train, image_nonoise_val = image_nonoise[:n_train], image_nonoise[n_train:]
label_train, label_val = label[:n_train], label[n_train:]
snr_train, snr_val = snr[:n_train], snr[n_train:]
sigma_train, sigma_val = sigma[:n_train], sigma[n_train:]
psf_train, psf_val = psf[:n_train], psf[n_train:]


# Zero-One Normalization of the labels
label_min = label.min(axis=0)
label_max = label.max(axis=0)
label_diff = np.maximum((label_max - label_min),1e-10)

label_train = (label_train - label_min)/label_diff
label_val = (label_val - label_min)/label_diff


# Zero-One Normalization of the no-noise images
image_nonoise_min = image_nonoise.min()
image_nonoise_max = image_nonoise.max()
image_nonoise_diff = (image_nonoise_max - image_nonoise_min)

image_nonoise_train = (image_nonoise_train - image_nonoise_min)/image_nonoise_diff
image_nonoise_val = (image_nonoise_val - image_nonoise_min)/image_nonoise_diff


# Zero-One Normalization of the noisy images
image_min = image.min()
image_max = image.max()
image_diff = (image_max - image_min)

image_train = (image_train - image_min)/image_diff
image_val = (image_val - image_min)/image_diff


# Compute Image Statistics
stats = np.hstack([image.mean(axis=(1,2)).reshape(-1,1), image.std(axis=(1,2)).reshape(-1,1), 
                   image.min(axis=(1,2)).reshape(-1,1), image.max(axis=(1,2)).reshape(-1,1)])
stats_train, stats_val = stats[:n_train], stats[n_train:]

### <div class='exercise'> <b> Step 3: Structure and Size of the Data </b> </div>

In [7]:
# Structure and Size of the Data
print("Image Shape            = {}".format(image.shape))
print("No-noise Image Shape   = {}".format(image_nonoise.shape))
print("Label Shape            = {}".format(label.shape))
print("SNR Shape              = {}".format(snr.shape))
print("Sigma Shape            = {}".format(sigma.shape))
print("PSF-R Shape            = {}".format(psf.shape))
print("Stats Shape            = {}".format(stats.shape))

Image Shape            = (200000, 64, 64, 1)
No-noise Image Shape   = (200000, 64, 64, 1)
Label Shape            = (200000, 7)
SNR Shape              = (200000,)
Sigma Shape            = (200000,)
PSF-R Shape            = (200000,)
Stats Shape            = (200000, 4)


### <div class='exercise'> <b> Step 4: Create Data Pipeline </b> </div>

<div class='gc'> <b> Create Tensorflow Dataset </b> </div>

In [8]:
# Create Tensorflow Dataset
training = tf.data.Dataset.from_tensor_slices({
    "Image": image_train,
    "No-noise Image": image_nonoise_train,
    "Labels": label_train,
    "Stats": stats_train
})

validation = tf.data.Dataset.from_tensor_slices({
    "Image": image_val,
    "No-noise Image": image_nonoise_val,
    "Labels": label_val,
    "Stats": stats_val
})

<div class='gc'> <b> Define pipeline functions to structure dataset </b> </div>

In [9]:
def pipeline_label_to_clean_image(element):
    image = element['No-noise Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    
    return ((label, stats), image)

In [10]:
def pipeline_noisy_image_to_label(element):
    image = element['Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    
    return ((image, stats), label)

In [11]:
def pipeline_noisy_image_to_clean_image(element):
    image = element['Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    clean = element['No-noise Image']
    
    return ((image, stats), (clean, label))

<div class='gc'> <b> Create dataset pipeline </b> </div>

In [12]:
# Batch Size
batch_size = 256


# Data pipeline for Decoder
tr_ds = training.map(
    pipeline_label_to_clean_image).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds = validation.map(
    pipeline_label_to_clean_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Data pipeline for Encoder
tr_ds_en = training.map(
    pipeline_noisy_image_to_label).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_en = validation.map(
    pipeline_noisy_image_to_label).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


# Data pipeline for AE
tr_ds_AE = training.map(
    pipeline_noisy_image_to_clean_image).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_AE = validation.map(
    pipeline_noisy_image_to_clean_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

### <div class='exercise'> <b> Step 5: Create Convolution Encoders and Decoder </b> </div>

<div class='gc'> <b> Declare global model parameters </b> </div>

In [13]:
K.clear_session()

# Encoder Inout Dimension
input_shape_img = (64,64,1)
input_shape_stats = (4,)

# Dropout rate for dense layers
dropout = 0.5

# Encoder Output dimension - Decoder Input Dimension
latent_z_dim_labels = 7
latent_z_dim_stats = 4

# Decoder latent dimension
latent_dim = (1,1,2048)

# Number of Training Epochs
epochs = 50

<div class='gc'> <b> Define convolution encoder </b> </div>

In [14]:
class ConvEncoder(layers.Layer):
    """
    Convolutional Encoder Layer Class.
    Converts an input into a latent representation.
    """

    def __init__(self, input_shape, dropout_rate=0.0, name='encoder', **kwargs):
        """
        Initializes the encoder layers and saves them as local attribute.
        
        Input:
        -input_dim: 3D-tuple with (rows, cols, channels) input image dimensions.
        
        Returns nothing.
        """
        super(ConvEncoder, self).__init__(name=name, input_shape=input_shape, **kwargs)
        
        ## your code here
        
        self.conv1 = layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', strides=(2,2))
        self.lRelu1 = layers.LeakyReLU()
        self.drop1 = layers.Dropout(dropout_rate)
        
        self.conv2 = layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', strides=(2,2))
        self.lRelu2 = layers.LeakyReLU()
        self.drop2 = layers.Dropout(dropout_rate)
        
        self.conv3 = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', strides=(2,2))
        self.lRelu3 = layers.LeakyReLU()
        self.drop3 = layers.Dropout(dropout_rate)
        
        self.conv4 = layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', strides=(2,2))
        self.lRelu4 = layers.LeakyReLU()
        self.drop4 = layers.Dropout(dropout_rate)
        
        self.conv5 = layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', strides=(2,2))
        self.lRelu5 = layers.LeakyReLU()
        self.drop5 = layers.Dropout(dropout_rate)
        
        self.conv6 = layers.Conv2D(filters=2048, kernel_size=(3,3), padding='same', strides=(2,2))
        self.lRelu6 = layers.LeakyReLU()
        self.drop6 = layers.Dropout(dropout_rate)
        
        # end of your code here
        

    def call(self, inputs, training=None):
        """
        Runs the encoding inference for `inputs`.
        
        Inputs:
        -inputs: 4D-tensor with dimension (batch_size, self.input_dim).
        """
        ## your code here
        
        z = self.conv1(inputs)
        z = self.lRelu1(z)
        z = self.drop1(z)
        
        z = self.conv2(z)
        z = self.lRelu2(z)
        z = self.drop2(z)
        
        z = self.conv3(z)
        z = self.lRelu3(z)
        z = self.drop3(z)
        
        z = self.conv4(z)
        z = self.lRelu4(z)
        z = self.drop4(z)
        
        z = self.conv5(z)
        z = self.lRelu5(z)
        z = self.drop5(z)
        
        z = self.conv6(z)
        z = self.lRelu6(z)
        z = self.drop6(z)
        
        # end of your code here
        return z

<div class='gc'> <b> Define convolution decoder </b> </div>

In [15]:
class ConvDecoder(layers.Layer):
    """
    Convolutional Decoder Layer Class.
    Converts z, the encoded digit vector, back into a readable digit.
    """

    def __init__(self, input_shape, dropout_rate=0.0, name='decoder', **kwargs):
        """
        Initializes the decoder architecture and saves it as a local attribute.
        
        Input:
        -input_shape: 3D-tuple with (rows, cols, channels) input representation.
        
        Returns nothing.
        """
        super(ConvDecoder, self).__init__(name=name, input_shape=input_shape, **kwargs)
        
        # your code here
        self.convT1 = layers.Conv2DTranspose(512, kernel_size=(3,3), strides=(2,2), padding='same')
        self.lRelu1 = layers.LeakyReLU()
        self.drop1 = layers.Dropout(dropout_rate)
        
        self.convT2 = layers.Conv2DTranspose(256, kernel_size=(3,3), strides=(2,2), padding='same')
        self.lRelu2 = layers.LeakyReLU()
        self.drop2 = layers.Dropout(dropout_rate)
        
        self.convT3 = layers.Conv2DTranspose(128, kernel_size=(3,3), strides=(2,2), padding='same')
        self.lRelu3 = layers.LeakyReLU()
        self.drop3 = layers.Dropout(dropout_rate)
        
        self.convT4 = layers.Conv2DTranspose(64, kernel_size=(3,3), strides=(2,2), padding='same')
        self.lRelu4 = layers.LeakyReLU()
        self.drop4 = layers.Dropout(dropout_rate)
        
        self.convT5 = layers.Conv2DTranspose(32, kernel_size=(3,3), strides=(2,2), padding='same')
        self.lRelu5 = layers.LeakyReLU()
        self.drop5 = layers.Dropout(dropout_rate)
        
        self.convT6 = layers.Conv2DTranspose(1, kernel_size=(3,3), strides=(2,2), padding='same', 
                                             activation='sigmoid')
        # end of your code here
        

    def call(self, inputs, training=None):
        """
        Runs the encoding inference for `inputs`.
        
        Inputs:
        -inputs: 4D-tensor with dimension (batch_size, self.input_dim).
        """
        ## your code here
        
        x = self.convT1(inputs)
        x = self.lRelu1(x)
        x = self.drop1(x)
        
        x = self.convT2(x)
        x = self.lRelu2(x)
        x = self.drop2(x)
        
        x = self.convT3(x)
        x = self.lRelu3(x)
        x = self.drop3(x)
        
        x = self.convT4(x)
        x = self.lRelu4(x)
        x = self.drop4(x)
        
        x = self.convT5(x)
        x = self.lRelu5(x)
        x = self.drop5(x)
        
        x = self.convT6(x)
        
        # end your code here
        return x

### <div class='exercise'> <b> Step 5: Create Conditional Auto-encoder </b> </div>

<div class='gc'> <b> Create Label to Image Decoder </b> </div>

In [16]:
# Create Reverse Decoder Model
def create_reverse_decoder():

    # Create Reverse Decoder
    input_decoder_labels = layers.Input(latent_z_dim_labels)
    #input_decoder_stats = layers.Input(latent_z_dim_stats)
    
    #x = layers.concatenate([input_decoder_labels, input_decoder_stats])
    
    x = layers.Dense(1024)(input_decoder_labels)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Dense(2048)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Reshape(latent_dim)(x)
    recon = ConvDecoder(latent_dim)(x)
    
    # Decoder Model
    #decoder = tf.keras.Model([input_decoder_labels, input_decoder_stats], recon, name='Reverse_Decoder')
    decoder = tf.keras.Model(input_decoder_labels, recon, name='Reverse_Decoder')
    display(decoder.summary())
    
    # Compile the model
    optimizer = optimizers.Adam(learning_rate=0.001)
    decoder.compile(optimizer, loss=losses.BinaryCrossentropy())
    
    # Return reverse decoder
    return decoder

reverse_decoder = create_reverse_decoder()

Model: "Reverse_Decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              8192      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              2099200   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
reshape (Reshape)            (None, 1, 1, 2048)    

None

<div class='gc'> <b> Create Image to Label Encoder conditional on Images Statistics</b> </div>

In [17]:
# Create Reverse Decoder Model
def create_reverse_encoder():

    # Create Reverse Encoder
    input_encoder_img = layers.Input(input_shape_img)
    input_encoder_stats = layers.Input(input_shape_stats)
    
    x = ConvEncoder(input_shape_img)(input_encoder_img)
    x = layers.Flatten()(x)
    
    x = layers.concatenate([x, input_encoder_stats])
    
    x = layers.Dense(2048)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Dense(1024)(x)
    x = layers.Dropout(dropout)(x)
    
    latent_z = layers.Dense(latent_z_dim_labels, activation='sigmoid')(x)
    
    
    # Encoder Model
    encoder = tf.keras.Model([input_encoder_img, input_encoder_stats], latent_z, name='Reverse_Encoder')
    display(encoder.summary())
    
    # Compile the model
    optimizer = optimizers.Adam(learning_rate=0.001)
    encoder.compile(optimizer, loss=losses.BinaryCrossentropy())
    
    # Return reverse decoder
    return encoder

reverse_encoder = create_reverse_encoder()

Model: "Reverse_Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
encoder (ConvEncoder)           (None, 1, 1, 2048)   11007232    input_2[0][0]                    
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           encoder[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 4)]          0                                            
____________________________________________________________________________________

None

<div class='gc'> <b> Create Noisy image to Clean image and Label Auto-encoder conditional on image statistics</b> </div>

In [18]:
def create_reverse_AE():
    input_encoder_img = layers.Input(input_shape_img)
    input_encoder_stats = layers.Input(input_shape_stats)
    
    labels = reverse_encoder([input_encoder_img, input_encoder_stats])
    recons = reverse_decoder(labels)
    
    # Reverse AE Model
    reverse_AE = tf.keras.Model([input_encoder_img, input_encoder_stats], [recons, labels], name='Reverse_AE')
    display(reverse_AE.summary())
    
    # Compile the model
    optimizer = optimizers.Adam(learning_rate=0.0001)
    reverse_AE.compile(optimizer, loss=losses.BinaryCrossentropy())
    
    # Return reverse decoder
    return reverse_AE

reverse_AE = create_reverse_AE()

Model: "Reverse_AE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
Reverse_Encoder (Model)         (None, 7)            17317127    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
Reverse_Decoder (Model)         (None, 64, 64, 1)    13112577    Reverse_Encoder[1][0]   

None

### <div class='exercise'> <b> Step 6: Training Conditional Auto-encoder on Image Dataset with fixed noise and PSF</b> </div>

<div class='gc'> <b> Traning conditional decoder</b> </div>

In [ ]:
# Run the model
try:
    reverse_decoder.load_weights('./model/rev_de5_1')
    print("Training Loss:   {:0.5f}".format(reverse_decoder.evaluate(tr_ds)))
    print("Validation Loss: {:0.5f}".format(reverse_decoder.evaluate(val_ds)))
    
except:
    history = reverse_decoder.fit(
        tr_ds, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_decoder.save_weights('./model/rev_de5_1')

632/704 [=========================>....] - ETA: 12s - loss: 0.0120

<div class='gc'> <b> Generating reconstructed images from labels for visual inspection </b> </div>

In [ ]:
test = val_ds.take(1)
labels, images = next(iter(test))
pred = reverse_decoder.predict(labels)
pred = pred.reshape(pred.shape[:-1])

In [ ]:
idx = np.random.randint(256*1, size=8)

fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Generated Images", fontsize=25)
for j in range(8):
    ax[j].imshow(pred[idx[j]])
plt.show()


fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Original Images", fontsize=25)
for j in range(8):
    ax[j].imshow(images[idx[j]].numpy().reshape(64,64))
plt.show()

<div class='gc'> <b> Traning conditional encoder</b> </div>

In [ ]:
# Run the model
try:
    reverse_encoder.load_weights('./model/rev_en5_1')
    print("Training Loss:   {:0.5f}".format(reverse_encoder.evaluate(tr_ds_en)))
    print("Validation Loss: {:0.5f}".format(reverse_encoder.evaluate(val_ds_en)))
    
except:
    history = reverse_encoder.fit(
        tr_ds_en, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds_en,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_encoder.save_weights('./model/rev_en5_1')

<div class='gc'> <b> Generating labels from noisy images for visual inspection </b> </div>

In [ ]:
test = val_ds_en.take(1)
images, labels = next(iter(test))
pred = reverse_encoder.predict(images)

In [ ]:
np.set_printoptions(suppress=True)

for i in idx:
    print("Pred:    ", tuple(pred[i]))
    print("Orignal: ", tuple(labels[i].numpy()))
    print("Error:   ", tuple(pred[i]-labels[i].numpy()))
    print("\n###########################################################################################\n")

<div class='gc'> <b> Variance Analysis </b> </div>

In [ ]:
pred = reverse_encoder.predict(val_ds_en)
error_unscaled = (pred - label_val) * label_diff

MSE = (error_unscaled**2).mean(axis=0)
RMSE = np.sqrt(MSE)

label_var = (label_val.std(axis=0) * label_diff)**2

unexplained_var = MSE/np.maximum(label_var, 1e-10)

print("RMSE:                  {}".format(tuple(RMSE)))
print("Unexplained Variance:  {}".format(tuple(unexplained_var)))
print("Explained Variance:    {}".format(tuple(1-unexplained_var)))

<div class='gc'> <b> Finetuning Conditional Auto-encoder </b> </div>

In [ ]:
# Run the model
try:
    reverse_AE.load_weights('./model/rev_AE5_1')
    print("Training Loss:   {}".format(reverse_AE.evaluate(tr_ds_AE)))
    print("Validation Loss: {}".format(reverse_AE.evaluate(val_ds_AE)))
    
except:
    history = reverse_AE.fit(
        tr_ds_AE, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds_AE,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_AE.save_weights('./model/rev_AE5_1')

<div class='gc'> <b> Generating reconstructed images from labels for visual inspection </b> </div>

In [ ]:
test = val_ds_AE.take(1)
(images, stats), (clean, labels) = next(iter(test))
pred_img, pred_lab = reverse_AE.predict([images, stats])
pred = pred_img.reshape(pred_img.shape[:-1])

In [ ]:
fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Generated Images", fontsize=25)
for j in range(8):
    ax[j].imshow(pred[idx[j]])
plt.show()


fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Clean Images", fontsize=25)
for j in range(8):
    ax[j].imshow(clean[idx[j]].numpy().reshape(64,64))
plt.show()

fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Original Noisy Images", fontsize=25)
for j in range(8):
    ax[j].imshow(images[idx[j]].numpy().reshape(64,64))
plt.show()

<div class='gc'> <b> Generating labels from noisy images for visual inspection </b> </div>

In [ ]:
for i in idx:
    print("Pred:    ", tuple(pred_lab[i]))
    print("Orignal: ", tuple(labels[i].numpy()))
    print("Error:   ", tuple(pred_lab[i]-labels[i].numpy()))
    print("\n###########################################################################################\n")

<div class='gc'> <b> Variance Analysis </b> </div>

In [ ]:
_, pred = reverse_AE.predict(val_ds_en)
error_unscaled = (pred - label_val) * label_diff

MSE = (error_unscaled**2).mean(axis=0)
RMSE = np.sqrt(MSE)

label_var = (label_val.std(axis=0) * label_diff)**2

unexplained_var = MSE/np.maximum(label_var, 1e-10)

print("RMSE:                  {}".format(tuple(RMSE)))
print("Unexplained Variance:  {}".format(tuple(unexplained_var)))
print("Explained Variance:    {}".format(tuple(1-unexplained_var)))

## <div class="theme"> End of Part 1 </div> 

## <div class="theme"> Part 2 </div> 

<div style="border: 3px solid #800080; padding: 10px"> 

## GalSim Images with variable Sigma and fixed PSF

### <div class='exercise'> <b> Step 1: Load the dataset with variable noise and fixed PSF </b> </div>

In [ ]:
# Read the data set
dataset = "./cs109b-project/data/data_v2.npz"

with np.load(dataset) as data:
    print("Available variables:", data.files)

    image = data["img"]
    image_nonoise = data["img_nonoise"]
    label = data["label"]
    snr = data["snr"]
    sigma = data["sigma"]
    psf = data["psf_r"]

<div class='gc'> <b> Join Sigma and PSF with the Label data </b> </div>

In [ ]:
# Combine Singma and PSF with the label
label = np.hstack([label, sigma.reshape(-1,1), psf.reshape(-1,1)])

### <div class='exercise'> <b> Step 2: Normalize the data and split the data into Traning and Validation sets </b> </div>

In [ ]:
# Reshape images to 3D array
image = image.reshape(image.shape[0], image.shape[1], image.shape[2], 1)
image_nonoise = image_nonoise.reshape(image_nonoise.shape[0], image_nonoise.shape[1], image_nonoise.shape[2], 1)


# Split into 90% train and 10% test (creates a view of the array)
n_train = int(label.shape[0] * 0.9)
image_train, image_val = image[:n_train], image[n_train:]
image_nonoise_train, image_nonoise_val = image_nonoise[:n_train], image_nonoise[n_train:]
label_train, label_val = label[:n_train], label[n_train:]
snr_train, snr_val = snr[:n_train], snr[n_train:]
sigma_train, sigma_val = sigma[:n_train], sigma[n_train:]
psf_train, psf_val = psf[:n_train], psf[n_train:]


# Zero-One Normalization of the labels
label_min = label.min(axis=0)
label_max = label.max(axis=0)
label_diff = np.maximum((label_max - label_min),1e-10)

label_train = (label_train - label_min)/label_diff
label_val = (label_val - label_min)/label_diff


# Zero-One Normalization of the no-noise images
image_nonoise_min = image_nonoise.min()
image_nonoise_max = image_nonoise.max()
image_nonoise_diff = (image_nonoise_max - image_nonoise_min)

image_nonoise_train = (image_nonoise_train - image_nonoise_min)/image_nonoise_diff
image_nonoise_val = (image_nonoise_val - image_nonoise_min)/image_nonoise_diff


# Zero-One Normalization of the noisy images
image_min = image.min()
image_max = image.max()
image_diff = (image_max - image_min)

image_train = (image_train - image_min)/image_diff
image_val = (image_val - image_min)/image_diff


# Compute Image Statistics
stats = np.hstack([image.mean(axis=(1,2)).reshape(-1,1), image.std(axis=(1,2)).reshape(-1,1), 
                   image.min(axis=(1,2)).reshape(-1,1), image.max(axis=(1,2)).reshape(-1,1)])
stats_train, stats_val = stats[:n_train], stats[n_train:]

### <div class='exercise'> <b> Step 3: Structure and Size of the Data </b> </div>

In [ ]:
# Structure and Size of the Data
print("Image Shape            = {}".format(image.shape))
print("No-noise Image Shape   = {}".format(image_nonoise.shape))
print("Label Shape            = {}".format(label.shape))
print("SNR Shape              = {}".format(snr.shape))
print("Sigma Shape            = {}".format(sigma.shape))
print("PSF-R Shape            = {}".format(psf.shape))
print("Stats Shape            = {}".format(stats.shape))

### <div class='exercise'> <b> Step 4: Create Data Pipeline </b> </div>

<div class='gc'> <b> Create Tensorflow Dataset </b> </div>

In [ ]:
# Create Tensorflow Dataset
training = tf.data.Dataset.from_tensor_slices({
    "Image": image_train,
    "No-noise Image": image_nonoise_train,
    "Labels": label_train,
    "Stats": stats_train
})

validation = tf.data.Dataset.from_tensor_slices({
    "Image": image_val,
    "No-noise Image": image_nonoise_val,
    "Labels": label_val,
    "Stats": stats_val
})

<div class='gc'> <b> Define pipeline functions to structure dataset </b> </div>

In [ ]:
def pipeline_label_to_clean_image(element):
    image = element['No-noise Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    
    return ((label, stats), image)

In [ ]:
def pipeline_noisy_image_to_label(element):
    image = element['Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    
    return ((image, stats), label)

In [ ]:
def pipeline_noisy_image_to_clean_image(element):
    image = element['Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    clean = element['No-noise Image']
    
    return ((image, stats), (clean, label))

<div class='gc'> <b> Create dataset pipeline </b> </div>

In [ ]:
# Batch Size
batch_size = 256


# Data pipeline for Decoder
tr_ds = training.map(
    pipeline_label_to_clean_image).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds = validation.map(
    pipeline_label_to_clean_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Data pipeline for Encoder
tr_ds_en = training.map(
    pipeline_noisy_image_to_label).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_en = validation.map(
    pipeline_noisy_image_to_label).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


# Data pipeline for AE
tr_ds_AE = training.map(
    pipeline_noisy_image_to_clean_image).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_AE = validation.map(
    pipeline_noisy_image_to_clean_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

### <div class='exercise'> <b> Step 6: Training Conditional Auto-encoder on Image Dataset with variable noise and fixed PSF</b> </div>

<div class='gc'> <b> Traning conditional decoder</b> </div>

In [ ]:
# Run the model
try:
    reverse_decoder.load_weights('./model/rev_de5_2')
    print("Training Loss:   {:0.5f}".format(reverse_decoder.evaluate(tr_ds)))
    print("Validation Loss: {:0.5f}".format(reverse_decoder.evaluate(val_ds)))
    
except:
    history = reverse_decoder.fit(
        tr_ds, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_decoder.save_weights('./model/rev_de5_2')

<div class='gc'> <b> Generating reconstructed images from labels for visual inspection </b> </div>

In [ ]:
test = val_ds.take(1)
labels, images = next(iter(test))
pred = reverse_decoder.predict(labels)
pred = pred.reshape(pred.shape[:-1])

In [ ]:
idx = np.random.randint(256*1, size=8)

fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Generated Images", fontsize=25)
for j in range(8):
    ax[j].imshow(pred[idx[j]])
plt.show()


fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Original Images", fontsize=25)
for j in range(8):
    ax[j].imshow(images[idx[j]].numpy().reshape(64,64))
plt.show()

<div class='gc'> <b> Traning conditional encoder</b> </div>

In [ ]:
# Run the model
try:
    reverse_encoder.load_weights('./model/rev_en5_2')
    print("Training Loss:   {:0.5f}".format(reverse_encoder.evaluate(tr_ds_en)))
    print("Validation Loss: {:0.5f}".format(reverse_encoder.evaluate(val_ds_en)))
    
except:
    history = reverse_encoder.fit(
        tr_ds_en, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds_en,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_encoder.save_weights('./model/rev_en5_2')

<div class='gc'> <b> Generating labels from noisy images for visual inspection </b> </div>

In [ ]:
test = val_ds_en.take(1)
images, labels = next(iter(test))
pred = reverse_encoder.predict(images)

In [ ]:
np.set_printoptions(suppress=True)

for i in idx:
    print("Pred:    ", tuple(pred[i]))
    print("Orignal: ", tuple(labels[i].numpy()))
    print("Error:   ", tuple(pred[i]-labels[i].numpy()))
    print("\n###########################################################################################\n")

<div class='gc'> <b> Variance Analysis </b> </div>

In [ ]:
pred = reverse_encoder.predict(val_ds_en)
error_unscaled = (pred - label_val) * label_diff

MSE = (error_unscaled**2).mean(axis=0)
RMSE = np.sqrt(MSE)

label_var = (label_val.std(axis=0) * label_diff)**2

unexplained_var = MSE/np.maximum(label_var, 1e-10)

print("RMSE:                  {}".format(tuple(RMSE)))
print("Unexplained Variance:  {}".format(tuple(unexplained_var)))
print("Explained Variance:    {}".format(tuple(1-unexplained_var)))

<div class='gc'> <b> Finetuning Conditional Auto-encoder </b> </div>

In [ ]:
# Run the model
try:
    reverse_AE.load_weights('./model/rev_AE5_2')
    print("Training Loss:   {}".format(reverse_AE.evaluate(tr_ds_AE)))
    print("Validation Loss: {}".format(reverse_AE.evaluate(val_ds_AE)))
    
except:
    history = reverse_AE.fit(
        tr_ds_AE, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds_AE,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_AE.save_weights('./model/rev_AE5_2')

<div class='gc'> <b> Generating reconstructed images from labels for visual inspection </b> </div>

In [ ]:
test = val_ds_AE.take(1)
(images, stats), (clean, labels) = next(iter(test))
pred_img, pred_lab = reverse_AE.predict([images, stats])
pred = pred_img.reshape(pred_img.shape[:-1])

In [ ]:
fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Generated Images", fontsize=25)
for j in range(8):
    ax[j].imshow(pred[idx[j]])
plt.show()


fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Clean Images", fontsize=25)
for j in range(8):
    ax[j].imshow(clean[idx[j]].numpy().reshape(64,64))
plt.show()

fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Original Noisy Images", fontsize=25)
for j in range(8):
    ax[j].imshow(images[idx[j]].numpy().reshape(64,64))
plt.show()

<div class='gc'> <b> Generating labels from noisy images for visual inspection </b> </div>

In [ ]:
for i in idx:
    print("Pred:    ", tuple(pred_lab[i]))
    print("Orignal: ", tuple(labels[i].numpy()))
    print("Error:   ", tuple(pred_lab[i]-labels[i].numpy()))
    print("\n###########################################################################################\n")

<div class='gc'> <b> Variance Analysis </b> </div>

In [ ]:
_, pred = reverse_AE.predict(val_ds_en)
error_unscaled = (pred - label_val) * label_diff

MSE = (error_unscaled**2).mean(axis=0)
RMSE = np.sqrt(MSE)

label_var = (label_val.std(axis=0) * label_diff)**2

unexplained_var = MSE/np.maximum(label_var, 1e-10)

print("RMSE:                  {}".format(tuple(RMSE)))
print("Unexplained Variance:  {}".format(tuple(unexplained_var)))
print("Explained Variance:    {}".format(tuple(1-unexplained_var)))

## <div class="theme"> End of Part 2 </div> 

## <div class="theme"> Part 3 </div> 

<div style="border: 3px solid #800080; padding: 10px"> 

## GalSim Images with variable Sigma and variable PSF

### <div class='exercise'> <b> Step 1: Load the dataset with variable noise and variable PSF </b> </div>

In [ ]:
# Read the data set
dataset = "./cs109b-project/data/data_v3.npz"

with np.load(dataset) as data:
    print("Available variables:", data.files)

    image = data["img"]
    image_nonoise = data["img_nonoise"]
    label = data["label"]
    snr = data["snr"]
    sigma = data["sigma"]
    psf = data["psf_r"]

<div class='gc'> <b> Join Sigma and PSF with the Label data </b> </div>

In [ ]:
# Combine Singma and PSF with the label
label = np.hstack([label, sigma.reshape(-1,1), psf.reshape(-1,1)])

### <div class='exercise'> <b> Step 2: Normalize the data and split the data into Traning and Validation sets </b> </div>

In [ ]:
# Reshape images to 3D array
image = image.reshape(image.shape[0], image.shape[1], image.shape[2], 1)
image_nonoise = image_nonoise.reshape(image_nonoise.shape[0], image_nonoise.shape[1], image_nonoise.shape[2], 1)


# Split into 90% train and 10% test (creates a view of the array)
n_train = int(label.shape[0] * 0.9)
image_train, image_val = image[:n_train], image[n_train:]
image_nonoise_train, image_nonoise_val = image_nonoise[:n_train], image_nonoise[n_train:]
label_train, label_val = label[:n_train], label[n_train:]
snr_train, snr_val = snr[:n_train], snr[n_train:]
sigma_train, sigma_val = sigma[:n_train], sigma[n_train:]
psf_train, psf_val = psf[:n_train], psf[n_train:]


# Zero-One Normalization of the labels
label_min = label.min(axis=0)
label_max = label.max(axis=0)
label_diff = np.maximum((label_max - label_min),1e-10)

label_train = (label_train - label_min)/label_diff
label_val = (label_val - label_min)/label_diff


# Zero-One Normalization of the no-noise images
image_nonoise_min = image_nonoise.min()
image_nonoise_max = image_nonoise.max()
image_nonoise_diff = (image_nonoise_max - image_nonoise_min)

image_nonoise_train = (image_nonoise_train - image_nonoise_min)/image_nonoise_diff
image_nonoise_val = (image_nonoise_val - image_nonoise_min)/image_nonoise_diff


# Zero-One Normalization of the noisy images
image_min = image.min()
image_max = image.max()
image_diff = (image_max - image_min)

image_train = (image_train - image_min)/image_diff
image_val = (image_val - image_min)/image_diff


# Compute Image Statistics
stats = np.hstack([image.mean(axis=(1,2)).reshape(-1,1), image.std(axis=(1,2)).reshape(-1,1), 
                   image.min(axis=(1,2)).reshape(-1,1), image.max(axis=(1,2)).reshape(-1,1)])
stats_train, stats_val = stats[:n_train], stats[n_train:]

### <div class='exercise'> <b> Step 3: Structure and Size of the Data </b> </div>

In [ ]:
# Structure and Size of the Data
print("Image Shape            = {}".format(image.shape))
print("No-noise Image Shape   = {}".format(image_nonoise.shape))
print("Label Shape            = {}".format(label.shape))
print("SNR Shape              = {}".format(snr.shape))
print("Sigma Shape            = {}".format(sigma.shape))
print("PSF-R Shape            = {}".format(psf.shape))
print("Stats Shape            = {}".format(stats.shape))

### <div class='exercise'> <b> Step 4: Create Data Pipeline </b> </div>

<div class='gc'> <b> Create Tensorflow Dataset </b> </div>

In [ ]:
# Create Tensorflow Dataset
training = tf.data.Dataset.from_tensor_slices({
    "Image": image_train,
    "No-noise Image": image_nonoise_train,
    "Labels": label_train,
    "Stats": stats_train
})

validation = tf.data.Dataset.from_tensor_slices({
    "Image": image_val,
    "No-noise Image": image_nonoise_val,
    "Labels": label_val,
    "Stats": stats_val
})

<div class='gc'> <b> Define pipeline functions to structure dataset </b> </div>

In [ ]:
def pipeline_label_to_clean_image(element):
    image = element['No-noise Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    
    return ((label, stats), image)

In [ ]:
def pipeline_noisy_image_to_label(element):
    image = element['Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    
    return ((image, stats), label)

In [ ]:
def pipeline_noisy_image_to_clean_image(element):
    image = element['Image']
    label = tf.cast(element['Labels'], tf.float32)
    stats = tf.cast(element['Stats'], tf.float32)
    clean = element['No-noise Image']
    
    return ((image, stats), (clean, label))

<div class='gc'> <b> Create dataset pipeline </b> </div>

In [ ]:
# Batch Size
batch_size = 256


# Data pipeline for Decoder
tr_ds = training.map(
    pipeline_label_to_clean_image).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds = validation.map(
    pipeline_label_to_clean_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Data pipeline for Encoder
tr_ds_en = training.map(
    pipeline_noisy_image_to_label).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_en = validation.map(
    pipeline_noisy_image_to_label).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


# Data pipeline for AE
tr_ds_AE = training.map(
    pipeline_noisy_image_to_clean_image).shuffle(1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_ds_AE = validation.map(
    pipeline_noisy_image_to_clean_image).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

### <div class='exercise'> <b> Step 6: Training Conditional Auto-encoder on Image Dataset with variable noise and PSF</b> </div>

<div class='gc'> <b> Traning conditional decoder</b> </div>

In [ ]:
# Run the model
try:
    reverse_decoder.load_weights('./model/rev_de5_3')
    print("Training Loss:   {:0.5f}".format(reverse_decoder.evaluate(tr_ds)))
    print("Validation Loss: {:0.5f}".format(reverse_decoder.evaluate(val_ds)))
    
except:
    history = reverse_decoder.fit(
        tr_ds, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_decoder.save_weights('./model/rev_de5_3')

<div class='gc'> <b> Generating reconstructed images from labels for visual inspection </b> </div>

In [ ]:
test = val_ds.take(1)
labels, images = next(iter(test))
pred = reverse_decoder.predict(labels)
pred = pred.reshape(pred.shape[:-1])

In [ ]:
idx = np.random.randint(256*1, size=8)

fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Generated Images", fontsize=25)
for j in range(8):
    ax[j].imshow(pred[idx[j]])
plt.show()


fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Original Images", fontsize=25)
for j in range(8):
    ax[j].imshow(images[idx[j]].numpy().reshape(64,64))
plt.show()

<div class='gc'> <b> Traning conditional encoder</b> </div>

In [ ]:
# Run the model
try:
    reverse_encoder.load_weights('./model/rev_en5_3')
    print("Training Loss:   {:0.5f}".format(reverse_encoder.evaluate(tr_ds_en)))
    print("Validation Loss: {:0.5f}".format(reverse_encoder.evaluate(val_ds_en)))
    
except:
    history = reverse_encoder.fit(
        tr_ds_en, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds_en,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_encoder.save_weights('./model/rev_en5_3')

<div class='gc'> <b> Generating labels from noisy images for visual inspection </b> </div>

In [ ]:
test = val_ds_en.take(1)
images, labels = next(iter(test))
pred = reverse_encoder.predict(images)

In [ ]:
np.set_printoptions(suppress=True)

for i in idx:
    print("Pred:    ", tuple(pred[i]))
    print("Orignal: ", tuple(labels[i].numpy()))
    print("Error:   ", tuple(pred[i]-labels[i].numpy()))
    print("\n###########################################################################################\n")

<div class='gc'> <b> Variance Analysis </b> </div>

In [ ]:
pred = reverse_encoder.predict(val_ds_en)
error_unscaled = (pred - label_val) * label_diff

MSE = (error_unscaled**2).mean(axis=0)
RMSE = np.sqrt(MSE)

label_var = (label_val.std(axis=0) * label_diff)**2

unexplained_var = MSE/np.maximum(label_var, 1e-10)

print("RMSE:                  {}".format(tuple(RMSE)))
print("Unexplained Variance:  {}".format(tuple(unexplained_var)))
print("Explained Variance:    {}".format(tuple(1-unexplained_var)))

<div class='gc'> <b> Finetuning Conditional Auto-encoder </b> </div>

In [ ]:
# Run the model
try:
    reverse_AE.load_weights('./model/rev_AE5_3')
    print("Training Loss:   {}".format(reverse_AE.evaluate(tr_ds_AE)))
    print("Validation Loss: {}".format(reverse_AE.evaluate(val_ds_AE)))
    
except:
    history = reverse_AE.fit(
        tr_ds_AE, 
        epochs=epochs, 
        verbose=1,
        validation_data=val_ds_AE,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
    )
    
    reverse_AE.save_weights('./model/rev_AE5_3')

<div class='gc'> <b> Generating reconstructed images from labels for visual inspection </b> </div>

In [ ]:
test = val_ds_AE.take(1)
(images, stats), (clean, labels) = next(iter(test))
pred_img, pred_lab = reverse_AE.predict([images, stats])
pred = pred_img.reshape(pred_img.shape[:-1])

In [ ]:
fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Generated Images", fontsize=25)
for j in range(8):
    ax[j].imshow(pred[idx[j]])
plt.show()


fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Clean Images", fontsize=25)
for j in range(8):
    ax[j].imshow(clean[idx[j]].numpy().reshape(64,64))
plt.show()

fig, ax = plt.subplots(1,8, figsize=(20,3))
plt.suptitle("Original Noisy Images", fontsize=25)
for j in range(8):
    ax[j].imshow(images[idx[j]].numpy().reshape(64,64))
plt.show()

<div class='gc'> <b> Generating labels from noisy images for visual inspection </b> </div>

In [ ]:
for i in idx:
    print("Pred:    ", tuple(pred_lab[i]))
    print("Orignal: ", tuple(labels[i].numpy()))
    print("Error:   ", tuple(pred_lab[i]-labels[i].numpy()))
    print("\n###########################################################################################\n")

<div class='gc'> <b> Variance Analysis </b> </div>

In [ ]:
_, pred = reverse_AE.predict(val_ds_en)
error_unscaled = (pred - label_val) * label_diff

MSE = (error_unscaled**2).mean(axis=0)
RMSE = np.sqrt(MSE)

label_var = (label_val.std(axis=0) * label_diff)**2

unexplained_var = MSE/np.maximum(label_var, 1e-10)

print("RMSE:                  {}".format(tuple(RMSE)))
print("Unexplained Variance:  {}".format(tuple(unexplained_var)))
print("Explained Variance:    {}".format(tuple(1-unexplained_var)))

## <div class="theme"> End of Part 3 </div> 

## <div class="theme"> Summary </div> 

<div style="border: 3px solid #800080; padding: 10px"> 
    
### ***Shape and Brightness of a Galaxy***


# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png">  End Of Project
